# Setup

## Libraries + dataset 

In [12]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf
import openpyxl

username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
            complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
        pathPart1 = "/Users/"
        pathPart2 = "/documents/Github/SCGProject/Datasets/"
        complePath = pathPart1+username+pathPart2

        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
            complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2
    complePath = '/home/alinux/unibg/git_repo/SCGProject/Datasets/'

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")

else:
    complePath = "C:/SCGProject/Datasets/"

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")


dfCambio["TassoCambioMedio"] = dfCambio["TassoCambioMedio"].str.replace(
    ',', '.')


Username:  marcovinciguerra


## Removing the intermediate in the dataset

### Budget

In [13]:
dfConsumi.rename(columns={'Budget/cons': 'budget'}, inplace=True)
qtaArtBudget = sqldf(
    'select NrArticolo, count(*) as qta from dfConsumi where budget = "BUDGET" group by NrArticolo')

dfVendite.rename(columns={'budget/cons': 'budget'}, inplace=True)
qtaArtVendBudget = sqldf(
    'select NrArticolo, count(*) as qta from dfVendite where budget = "BUDGET" group by NrArticolo')

artProdeVendBudget = sqldf(
    'select P.NrArticolo from qtaArtBudget P join qtaArtVendBudget V on P.NrArticolo = V.NrArticolo')


consumiB = sqldf('select * from dfConsumi where budget = "BUDGET"')


# ho trovato articoli che hanno consumato risorse a budget ma non sono stati messi tra le vendite a budget
artProdeNONVendBudget = sqldf(
    'select * from consumiB group by NrArticolo having NrArticolo not in (select NrArticolo from artProdeVendBudget)')

# prendo gli articoli che usano come materia prima almeno un articolo prodotto internamente
# sqldf('select NrMovimento, budget, C.NrArticolo, CodiceMP, NrDocumento, QuantityMPImpiegata, ImportoCostoTOTALE from consumiB C join artProdeNONVendBudget A on C.CodiceMP = A.NrArticolo')

costoPerArticoloBudget = sqldf(
    'select NrArticolo, sum(ImportoCostoTOTALE) as costoMpPerArt from consumiB where NrArticolo not in (select NrArticolo from artProdeNONVendBudget) group by NrArticolo')
sqldf('select sum(costoMpPerArt) as costoTotBudget from costoPerArticoloBudget')

mpB = sqldf('''select NrMovimento,budget,CodiceMP,NrArticolo, NrDocumento, sum(QuantityMPImpiegata), sum(ImportoCostoTOTALE) from consumiB group by NrArticolo''')
prodInternaB = sqldf('''select A.NrMovimento, A.budget, A.CodiceMP, A.NrArticolo, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE
         from consumiB A join mpB B on A.CodiceMP = B.NrArticolo''')

mpIntermediB = sqldf('''select distinct CodiceMP from prodInternaB''')
consumiBNOArtInterm = sqldf(
    '''select * from consumiB where NrArticolo not in (select CodiceMP from mpIntermediB)''')


### Consuntivo

In [14]:
consumiC = sqldf(
    '''select * from dfConsumi where budget = "CONSUNTIVO" or budget = "Consuntivo"''')
mpC = sqldf('''select NrMovimento,budget,CodiceMP,NrArticolo, NrDocumento, sum(QuantityMPImpiegata), sum(ImportoCostoTOTALE) from consumiC group by NrArticolo''')
#A.NrArticolo, A.CodiceMP, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE, A.CodiceMP
prodInternaC = sqldf('''select A.NrMovimento, A.budget, A.CodiceMP, A.NrArticolo, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE
         from consumiC A join mpC B on A.CodiceMP = B.NrArticolo''')
mpIntermediC = sqldf('''select distinct CodiceMP from prodInternaC''')
consumiCNOArtInterm = sqldf(
    '''select * from consumiC where NrArticolo NOT IN (select CodiceMP from mpIntermediC)''')


### UNIONE DELLE DUE TABELLE PER FARE IL NUOVO DATASET

In [15]:
dfConsumi = sqldf('''  SELECT *
                            FROM consumiBNOArtInterm
                            UNION
                            SELECT *
                            FROM consumiCNOArtInterm
                            ''')
del dfConsumi['Unnamed: 0']
dfConsumi


,NrMovimento,budget,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,28854,BUDGET,ART0001074,ART0000018,ODP17-05342,30.0,5.15
1,66819,CONSUNTIVO,ART0001074,ART0000018,ODP19-05998,30.0,5.15
2,28853,BUDGET,ART0005696,ART0000041,ODP17-05342,4.0,92.00
3,66818,CONSUNTIVO,ART0005696,ART0000041,ODP19-05998,4.0,88.00
4,28855,BUDGET,ART0005697,ART0000042,ODP17-05342,4.0,104.00
...,...,...,...,...,...,...,...
4046,66172,CONSUNTIVO,ART0005850,ART0005136,ODP19-02057,155.0,1.23
4047,66235,CONSUNTIVO,ART0006245,ART0005136,ODP19-02528,700.0,0.00
4048,66446,CONSUNTIVO,ART0006381,ART0005136,ODP19-02674,300.0,1.62
4049,66594,CONSUNTIVO,ART0006345,ART0005136,ODP19-06688,9.0,0.00


# Funzione che fa partire l'analisi degli scostamenti

## Reading delle tabelle che servono 

In [16]:
venditeS1   = pd.read_csv("venditeS1.csv")
MergeImpieg = pd.read_csv("MergeImpieg.csv")
ConsumiUnit = pd.read_csv("ConsumiUnit.csv")


## Dichiarazione della funzione

In [17]:
def by_articolo_full(venditeS1, mergeImpiego, ConsumiUnit, articolo):

        venditeSingoloArticolo = sqldf('select * from venditeS1 where NrArticolo = "'+articolo+'"')

        BudgetStandardRic = sqldf('select sum(Mix_y*VolumeTotale_y*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
        MixStandardRic = sqldf('select sum(Mix_y*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
        MixEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
        BudgetEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
        BudgetEffettivoValutaRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_x)/100000000000 from venditeSingoloArticolo ').iloc[0][0]

        if(BudgetStandardRic is None):
                BudgetStandardRic = 0

        if(MixStandardRic is None):
                MixStandardRic = 0

        if(MixEffettivoRic is None):
                MixEffettivoRic = 0

        if(BudgetEffettivoRic is None):
                BudgetEffettivoRic = 0

        if(BudgetEffettivoValutaRic is None):
                BudgetEffettivoValutaRic = 0

        ScostamentoVolumeRic = MixStandardRic-BudgetStandardRic
        ScostamentoMixRic = MixEffettivoRic-MixStandardRic
        ScostamentoPrezzoRic = BudgetEffettivoRic-MixEffettivoRic
        ScostamentoValutaRic = BudgetEffettivoValutaRic-BudgetEffettivoRic


        lavorazioniSingoloProdotto = sqldf('select * from x where NrArticolo = "'+articolo+'"')

        #Budget Standard
        BudgetStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_y) from lavorazioniSingoloProdotto').iloc[0][0]
        # Impiego Standard (volume a consuntivo)
        ImpiegoStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_x) from lavorazioniSingoloProdotto').iloc[0][0]
        # Impiego Effettivo (volume e impiego a consuntivo)
        ImpiegoEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_y*QuantitydiOutput_x) from lavorazioniSingoloProdotto').iloc[0][0]
        # Budget Effettivo (tutto a consuntivo)
        BudgetEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_x*QuantitydiOutput_x) from lavorazioniSingoloProdotto').iloc[0][0]

        if(BudgetStandardLav is None):
                        BudgetStandardLav = 0

        if(ImpiegoStandardLav is None):
                        ImpiegoStandardLav = 0

        if(ImpiegoEffettivoLav is None):
                        ImpiegoEffettivoLav = 0

        if(BudgetEffettivoLav is None):
                        BudgetEffettivoLav = 0

        #Scostamenti
        ScostamentoVolumeLav = ImpiegoStandardLav-BudgetStandardLav
        ScostamentoImpiegoLav = ImpiegoEffettivoLav-ImpiegoStandardLav
        ScostamentoPrezzoLav = BudgetEffettivoLav-ImpiegoEffettivoLav

        consumiSingoloArticolo = sqldf('select * from ConsumiUnit where NrArticolo="'+articolo+'"')

        BudgetEffettivoMP = sqldf('select sum(QuantitydiOutput_x*ImpiegoPerPezzo_x*CostoPerUnitaMis_x) from consumiSingoloArticolo').iloc[0][0]
        ImpiegoEffettivoMP = sqldf('select sum(QuantitydiOutput_x*ImpiegoPerPezzo_x*CostoPerUnitaMis_y) from consumiSingoloArticolo').iloc[0][0]
        ImpiegoStandardMP = sqldf('select sum(QuantitydiOutput_x*ImpiegoPerPezzo_y*CostoPerUnitaMis_y) from consumiSingoloArticolo').iloc[0][0]
        BudgetStandardMP = sqldf('select sum(QuantitydiOutput_y*ImpiegoPerPezzo_y*CostoPerUnitaMis_y) from consumiSingoloArticolo').iloc[0][0]

        #TODO: choose strategy 

        BudgetEffettivoMP = sqldf('select (avg(QuantitydiOutput_x)*avg(ImpiegoPerPezzo_x)*avg(CostoPerUnitaMis_x)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]
        ImpiegoEffettivoMP = sqldf('select (avg(QuantitydiOutput_x)*avg(ImpiegoPerPezzo_x)*avg(CostoPerUnitaMis_y)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]
        ImpiegoStandardMP = sqldf('select (avg(QuantitydiOutput_x)*avg(ImpiegoPerPezzo_y)*avg(CostoPerUnitaMis_y)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]
        BudgetStandardMP = sqldf('select (avg(QuantitydiOutput_y)*avg(ImpiegoPerPezzo_y)*avg(CostoPerUnitaMis_y)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]

        if(BudgetStandardMP is None):
                BudgetStandardMP = 0

        if(ImpiegoStandardMP is None):
                ImpiegoStandardMP = 0

        if(ImpiegoEffettivoMP is None):
                ImpiegoEffettivoMP = 0

        if(BudgetEffettivoMP is None):
                BudgetEffettivoMP = 0

        ScostamentoVolumeMP =  ImpiegoStandardMP - BudgetStandardMP
        ScostamentoImpiegoMP = ImpiegoEffettivoMP-ImpiegoStandardMP
        ScostamentoPrezzoMP = BudgetEffettivoMP-ImpiegoEffettivoMP 

        data = {articolo: ['Ricavi', 'CostiMP', 'CostiLav', 'Margine'], 
                'Budget' : [BudgetStandardRic, BudgetStandardMP,BudgetStandardLav, (BudgetStandardRic - (BudgetStandardMP+BudgetStandardLav))],
                'ScostamentoVolume' : [ScostamentoVolumeRic,ScostamentoVolumeMP,ScostamentoVolumeLav, (ScostamentoVolumeRic - (ScostamentoVolumeMP+ScostamentoVolumeLav))], 
                'MixStandard' : [MixStandardRic, ImpiegoStandardMP,ImpiegoStandardLav, (MixStandardRic - (ImpiegoStandardMP+ImpiegoStandardLav))],
                'ScostamentoMix' : [ScostamentoMixRic,ScostamentoImpiegoMP,ScostamentoImpiegoLav, (ScostamentoMixRic - (ScostamentoImpiegoMP+ScostamentoImpiegoLav))],
                'MixEffettivo' : [MixEffettivoRic, ImpiegoEffettivoMP,ImpiegoEffettivoLav, (MixEffettivoRic - (ImpiegoEffettivoMP+ImpiegoEffettivoLav))],
                'ScostamentoPrezzo' : [ScostamentoPrezzoRic,ScostamentoPrezzoMP,ScostamentoPrezzoLav, (ScostamentoPrezzoRic - (ScostamentoPrezzoMP+ScostamentoPrezzoLav))], 
                'MixValuta' : [BudgetEffettivoRic, BudgetEffettivoMP,BudgetEffettivoLav, (BudgetEffettivoRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
                'ScostamentoValuta' : [ScostamentoValutaRic, 0, 0, (ScostamentoValutaRic - (0+0))],
                'Consuntivo' : [BudgetEffettivoValutaRic, BudgetEffettivoMP, BudgetEffettivoLav, (BudgetEffettivoValutaRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
                'ScostamentoTotale' : [BudgetEffettivoValutaRic-BudgetStandardRic,BudgetEffettivoMP-BudgetStandardMP,BudgetEffettivoLav-BudgetStandardLav, (BudgetEffettivoValutaRic - BudgetStandardRic - (BudgetEffettivoMP-BudgetStandardMP+BudgetEffettivoLav-BudgetStandardLav))]}

        return pd.DataFrame(data)


In [19]:
by_articolo_full(venditeS1,MergeImpieg,ConsumiUnit,'ART0002240')

PandaSQLException: (sqlite3.OperationalError) no such table: x
[SQL: select * from x where NrArticolo = "ART0002240"]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
articolo = 'ART0002240'
articolo = 'ART0000308'
articolo = 'ART0005136'

mergeImpiego = x

venditeSingoloArticolo = sqldf('select * from venditeS1 where NrArticolo = "'+articolo+'"')

BudgetStandardRic = sqldf('select sum(Mix_y*VolumeTotale_y*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
MixStandardRic = sqldf('select sum(Mix_y*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
MixEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
BudgetEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_y)/100000000000 from venditeSingoloArticolo ').iloc[0][0]
BudgetEffettivoValutaRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_x)/100000000000 from venditeSingoloArticolo ').iloc[0][0]

if(BudgetStandardRic is None):
        BudgetStandardRic = 0

if(MixStandardRic is None):
        MixStandardRic = 0

if(MixEffettivoRic is None):
        MixEffettivoRic = 0

if(BudgetEffettivoRic is None):
        BudgetEffettivoRic = 0

if(BudgetEffettivoValutaRic is None):
        BudgetEffettivoValutaRic = 0

ScostamentoVolumeRic = MixStandardRic-BudgetStandardRic
ScostamentoMixRic = MixEffettivoRic-MixStandardRic
ScostamentoPrezzoRic = BudgetEffettivoRic-MixEffettivoRic
ScostamentoValutaRic = BudgetEffettivoValutaRic-BudgetEffettivoRic


lavorazioniSingoloProdotto = sqldf('select * from x where NrArticolo = "'+articolo+'"')

#Budget Standard
BudgetStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_y) from lavorazioniSingoloProdotto').iloc[0][0]
# Impiego Standard (volume a consuntivo)
ImpiegoStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_x) from lavorazioniSingoloProdotto').iloc[0][0]
# Impiego Effettivo (volume e impiego a consuntivo)
ImpiegoEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_y*QuantitydiOutput_x) from lavorazioniSingoloProdotto').iloc[0][0]
# Budget Effettivo (tutto a consuntivo)
BudgetEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_x*QuantitydiOutput_x) from lavorazioniSingoloProdotto').iloc[0][0]

if(BudgetStandardLav is None):
                BudgetStandardLav = 0

if(ImpiegoStandardLav is None):
                ImpiegoStandardLav = 0

if(ImpiegoEffettivoLav is None):
                ImpiegoEffettivoLav = 0

if(BudgetEffettivoLav is None):
                BudgetEffettivoLav = 0

#Scostamenti
ScostamentoVolumeLav = ImpiegoStandardLav-BudgetStandardLav
ScostamentoImpiegoLav = ImpiegoEffettivoLav-ImpiegoStandardLav
ScostamentoPrezzoLav = BudgetEffettivoLav-ImpiegoEffettivoLav

consumiSingoloArticolo = sqldf('select * from ConsumiUnit where NrArticolo="'+articolo+'"')

BudgetEffettivoMP = sqldf('select sum(QuantitydiOutput_x*ImpiegoPerPezzo_x*CostoPerUnitaMis_x) from consumiSingoloArticolo').iloc[0][0]
ImpiegoEffettivoMP = sqldf('select sum(QuantitydiOutput_x*ImpiegoPerPezzo_x*CostoPerUnitaMis_y) from consumiSingoloArticolo').iloc[0][0]
ImpiegoStandardMP = sqldf('select sum(QuantitydiOutput_x*ImpiegoPerPezzo_y*CostoPerUnitaMis_y) from consumiSingoloArticolo').iloc[0][0]
BudgetStandardMP = sqldf('select sum(QuantitydiOutput_y*ImpiegoPerPezzo_y*CostoPerUnitaMis_y) from consumiSingoloArticolo').iloc[0][0]

#TODO: choose strategy 

BudgetEffettivoMP = sqldf('select (avg(QuantitydiOutput_x)*avg(ImpiegoPerPezzo_x)*avg(CostoPerUnitaMis_x)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]
ImpiegoEffettivoMP = sqldf('select (avg(QuantitydiOutput_x)*avg(ImpiegoPerPezzo_x)*avg(CostoPerUnitaMis_y)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]
ImpiegoStandardMP = sqldf('select (avg(QuantitydiOutput_x)*avg(ImpiegoPerPezzo_y)*avg(CostoPerUnitaMis_y)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]
BudgetStandardMP = sqldf('select (avg(QuantitydiOutput_y)*avg(ImpiegoPerPezzo_y)*avg(CostoPerUnitaMis_y)) from consumiSingoloArticolo group by NrArticolo').iloc[0][0]

if(BudgetStandardMP is None):
        BudgetStandardMP = 0

if(ImpiegoStandardMP is None):
        ImpiegoStandardMP = 0

if(ImpiegoEffettivoMP is None):
        ImpiegoEffettivoMP = 0

if(BudgetEffettivoMP is None):
        BudgetEffettivoMP = 0

ScostamentoVolumeMP =  ImpiegoStandardMP - BudgetStandardMP
ScostamentoImpiegoMP = ImpiegoEffettivoMP-ImpiegoStandardMP
ScostamentoPrezzoMP = BudgetEffettivoMP-ImpiegoEffettivoMP 

data = {'Index': ['Ricavi', 'CostiMP', 'CostiLav', 'Margine'], 
        'Budget' : [BudgetStandardRic, BudgetStandardMP,BudgetStandardLav, (BudgetStandardRic - (BudgetStandardMP+BudgetStandardLav))],
        'ScostamentoVolume' : [ScostamentoVolumeRic,ScostamentoVolumeMP,ScostamentoVolumeLav, (ScostamentoVolumeRic - (ScostamentoVolumeMP+ScostamentoVolumeLav))], 
        'MixStandard' : [MixStandardRic, ImpiegoStandardMP,ImpiegoStandardLav, (MixStandardRic - (ImpiegoStandardMP+ImpiegoStandardLav))],
        'ScostamentoMix' : [ScostamentoMixRic,ScostamentoImpiegoMP,ScostamentoImpiegoLav, (ScostamentoMixRic - (ScostamentoImpiegoMP+ScostamentoImpiegoLav))],
        'MixEffettivo' : [MixEffettivoRic, ImpiegoEffettivoMP,ImpiegoEffettivoLav, (MixEffettivoRic - (ImpiegoEffettivoMP+ImpiegoEffettivoLav))],
        'ScostamentoPrezzo' : [ScostamentoPrezzoRic,ScostamentoPrezzoMP,ScostamentoPrezzoLav, (ScostamentoPrezzoRic - (ScostamentoPrezzoMP+ScostamentoPrezzoLav))], 
        'MixValuta' : [BudgetEffettivoRic, BudgetEffettivoMP,BudgetEffettivoLav, (BudgetEffettivoRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
        'ScostamentoValuta' : [ScostamentoValutaRic, 0, 0, (ScostamentoValutaRic - (0+0))],
        'Consuntivo' : [BudgetEffettivoValutaRic, BudgetEffettivoMP, BudgetEffettivoLav, (BudgetEffettivoValutaRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
        'ScostamentoTotale' : [BudgetEffettivoValutaRic-BudgetStandardRic,BudgetEffettivoMP-BudgetStandardMP,BudgetEffettivoLav-BudgetStandardLav, (BudgetEffettivoValutaRic - BudgetStandardRic - (BudgetEffettivoMP-BudgetStandardMP+BudgetEffettivoLav-BudgetStandardLav))]}

pd.DataFrame(data)


,Index,Budget,ScostamentoVolume,MixStandard,ScostamentoMix,MixEffettivo,ScostamentoPrezzo,MixValuta,ScostamentoValuta,Consuntivo,ScostamentoTotale
0,Ricavi,12932.919142,3768.520874,16701.440016,-7967.750215,8733.689801,2863.053503,11596.743303,-1094.60554,10502.137763,-2430.781379
1,CostiMP,694.226733,694.226733,1388.453465,1314.397385,2702.850851,16274.438521,18977.289371,0.00000,18977.289371,18283.062639
2,CostiLav,4877.141333,6460.386726,11337.528059,2081.779451,13419.307510,8881.716494,22301.024004,0.00000,22301.024004,17423.882671
3,Margine,7361.551076,-3386.092584,3975.458492,-11363.927052,-7388.468560,-22293.101512,-29681.570072,-1094.60554,-30776.175613,-38137.726689
